In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-


In [1]:
__author__ = "Tariq Faquih"
__copyright__ = "Copyright 2020, Clinical Epidemiology Department, LUMC"
__credits__ = ["Tariq Faquih"]
__license__ = "MIT License"
__maintainer__ = "Tariq Faquiih"
__email__ = "t.o.faquih@lumc.nl"
__status__ = "Development"

# Import modules

In [2]:
import json, csv , os , sys , datetime
from Bio import Entrez
from Bio import Medline
from datetime import datetime
from datetime import timedelta


# COVID class

This class objects send pubmed queries through the API and stores the output in json files

In [3]:

class COVID:
 
    def __init__(self , startD , endD):

        #list of query terms to be used in the search_function

        self.querydict = {'opioids':None , 
                 'Big Five':'(NEJM[journal] OR BMJ[journal] OR lancet[journal] OR nature[journal] OR JAMA[journal])', 
                 'Elderly':'elderly[TITLE]',
                 'Clinical Trial':'clinical trial[Title/Abstract]' , 
                 'Italy':'italy[Title/Abstract]' , 
                 'Netherlands':'netherlands[Title/Abstract]' , 
                 'Case Control':'case control study' , 
                 'Epidemiology':'epidemiology' , 
                 'Mortality':'mortality', 
                 'Pregnant':'pregnant[TITLE]' }
        
        #json_file stores the proper json format to be used in the googlesheet
        #dict_file stores the output in a dictionary to be loaded in later uses
        json_file ='../jsonfiles/opioids.json'
        dict_file ='../jsonfiles/opioids_dictionary4.json'

        #read the stored dictionary file (dict_file) or create a new blank dictionary
        if os.path.isfile(dict_file):
            self.mainDict = json.load(open(dict_file))
        else:
            self.mainDict = {}
        
        #set counter for how many articles are added and create a list to store the log messages
        self.NumNew = 0
        self.Log = []
        
        #for each query term in the querylist, run the search function using the provideed start
        #and end dates
        for K in self.querydict.keys():
            for DB in ('pubmed' , 'pmc'):
                self.search_function(K , DB , startD , endD )
            
        #for X in querylist:
        #    self.search_function(X , startD , endD )
        
        #add the total number of added articles to the log list
        self.Log.append(self.NumNew)
        
        #Write the main json file
        with open(json_file , 'w') as fp:   
            Output=[]
            for Key,Item in self.mainDict.items():
                Output.append(Item)
            json.dump(Output, fp)
            
        #Write the exact dict as json file (easily read by the script)
        with open(dict_file , 'w') as fp:   
            json.dump(self.mainDict, fp)
        
        with open('../logs/log_{}_{}.txt'.format(startD.replace('/' , '') , endD.replace('/' , '')) ,'w'  , newline='') as fp:
            W = csv.writer(fp)
            W.writerow(['Search results for range {} to {}'.format(startD , endD)])
            W.writerow(['Number of Records Added: {}'.format(self.Log[-1])])
            for Line in self.Log[:-1]:
                W.writerow([Line])
        

    def search_function (self , MyTerms, DB , startD , endD):

        Entrez.email = "tariqf549@gmail.com"
        MainTerm = """"analgesics, opioid"[Pharmacological Action] OR "analgesics, opioid"[MeSH Terms] OR ("analgesics"[All Fields] AND "opioid"[All Fields]) OR "opioid analgesics"[All Fields] OR "opioids"[All Fields]"""
        #MainTerm = '"COVID-19"[All Fields]'
        DateRange = '"{}"[PDAT] : "{}"[PDAT]'.format(startD , endD)
        if MyTerms == 'opioids':
            Query = MainTerm + ' AND ' + DateRange
        else:
            Query = MainTerm + ' AND ' + self.querydict[MyTerms] + ' AND ' + DateRange
            
        print(Query)
        self.Log.append(Query)
        search_results = Entrez.read(
            Entrez.esearch(
                db=DB, term=Query,  datetype="pdat", usehistory="y" , sort = 'relevance' 
            )
        )
        count = int(search_results["Count"])
        self.Log.append("Found %i results" % count)
        
        print("Found %i results" % count)

        batch_size = 10
        out_handle = open("../pubmed_results/opioids_{}_papers.txt".format(MyTerms), "w" , encoding="utf-8")
        for start in range(0, count, batch_size):
            end = min(count, start + batch_size)
            print("Going to download record %i to %i" % (start + 1, end))
            self.Log.append("Going to download record %i to %i" % (start + 1, end))
            fetch_handle = Entrez.efetch(
                db=DB,
                rettype="medline",
                retmode="text",
                retstart=start,
                retmax=batch_size,
                webenv=search_results["WebEnv"],
                query_key=search_results["QueryKey"],
            )
            data = fetch_handle.read()

            dataresults = data.split('\nPMID')[1:]
            self.add2dict(dataresults , MyTerms)
            #print(data)
            fetch_handle.close()
            out_handle.write(data)
        out_handle.close()
        
    def FormatAbstract (self, AB):
        Abstract = ''
        if 'AB' in AB.keys():
            Abstract = AB['AB']
            tempAbs = Abstract.split(' ')
            lastN=0
            newabs = []
            for N in range(30, len(tempAbs)+30, 30) :
                newabs.append(' '.join(tempAbs[lastN:N]))
                lastN= N
                
            Abstract = '\n'.join(newabs)
                
        else: 
            Abstract = 'NA'
            
        return(Abstract)
            
    def add2dict(self , dataresults , Q):
        for hit in dataresults:
            m1 = 'PMID' + hit
            parse_res = Medline.read(m1.split('\n'))
            if 'PMID' in parse_res.keys():
                PMID = parse_res['PMID']
                
            elif 'PMCID' in parse_res.keys():
                PMID = parse_res['PMCID']
                
            else: PMID = ''
                
            Tag = Q
            if PMID in self.mainDict.keys():
                print('PMID [{}] exists'.format(PMID))
                self.Log.append('PMID exists')
                if Q not in self.mainDict[PMID]['Tag']:
                    self.mainDict[PMID]['Tag'] = self.mainDict[PMID]['Tag']+' '+Tag
                    
                if self.mainDict[PMID]['Abstract'] == 'NA':
                    NewABS = self.FormatAbstract(parse_res)
                    if NewABS != 'NA':
                        self.mainDict[PMID]['Abstract'] = self.FormatAbstract(parse_res)
                        print('Updated Abstract')
                        self.Log.append('Updated Abstract')
                        

                    
                continue
            else:
                Title = parse_res['TI']
                
                if 'JT' in parse_res.keys():
                    JournalName  = parse_res['JT']
                else: JournalName = ''
                    
                #Date of Publication [dp] - Date searching includes both print and electronic dates of publication. 
                #Searching for a single date does not include items when the electronic date of publication is after the print date.
                if 'DP' in parse_res.keys():
                    dateP  = parse_res['DP']
                else: dateP = ''
                
                #The date the citation first entered PubMed. In PMC DEP is used instead
                if 'EDAT' in parse_res.keys():
                    dateC  = datetime.strptime(parse_res['EDAT'] , '%Y/%m/%d %H:%S')
                    dateC = dateC.strftime("%Y%m%d")
                elif 'DEP' in parse_res.keys():
                    dateC = parse_res['DEP']
                else: dateC = ''

                Abstract = self.FormatAbstract(parse_res)
                
                    
                Link= 'https://www.ncbi.nlm.nih.gov/pubmed/{}'.format(PMID)   


                self.mainDict[PMID] = {'PMID': PMID, 'Title':Title ,
                                'JournalName':JournalName ,
                                  'Date Added':dateC ,
                                'Publication Date':dateP , 
                                'Abstract':Abstract , 
                                'Link':Link,
                                'Tag':Tag   }
                self.NumNew +=1
                print('Added new PMID: {}'.format(PMID))
                self.Log.append('Added new PMID: {}'.format(PMID))


In [14]:
if __name__ == '__main__':
    Today = datetime.now()
    StartDate = Today - timedelta(days=3)

    Today = Today.strftime("%Y/%m/%d")
    StartDate = StartDate.strftime("%Y/%m/%d")
    
    COVID(StartDate , Today )

"analgesics, opioid"[Pharmacological Action] OR "analgesics, opioid"[MeSH Terms] OR ("analgesics"[All Fields] AND "opioid"[All Fields]) OR "opioid analgesics"[All Fields] OR "opioids"[All Fields] AND "2020/03/27"[PDAT] : "2020/03/30"[PDAT]
Found 7 results
Going to download record 1 to 7
PMID [32220099] exists
PMID [32221355] exists
PMID [32221867] exists
PMID [32219431] exists
PMID [32219415] exists
PMID [32220937] exists
PMID [32219920] exists
"analgesics, opioid"[Pharmacological Action] OR "analgesics, opioid"[MeSH Terms] OR ("analgesics"[All Fields] AND "opioid"[All Fields]) OR "opioid analgesics"[All Fields] OR "opioids"[All Fields] AND "2020/03/27"[PDAT] : "2020/03/30"[PDAT]
Found 4 results
Going to download record 1 to 4
"analgesics, opioid"[Pharmacological Action] OR "analgesics, opioid"[MeSH Terms] OR ("analgesics"[All Fields] AND "opioid"[All Fields]) OR "opioid analgesics"[All Fields] OR "opioids"[All Fields] AND (NEJM[journal] OR BMJ[journal] OR lancet[journal] OR nature[jou

FileNotFoundError: [Errno 2] No such file or directory: 'logs/log_20200327_20200330.txt'

In [4]:
COVID('2020/01/01' , '2020/03/30' )

"analgesics, opioid"[Pharmacological Action] OR "analgesics, opioid"[MeSH Terms] OR ("analgesics"[All Fields] AND "opioid"[All Fields]) OR "opioid analgesics"[All Fields] OR "opioids"[All Fields] AND "2020/01/01"[PDAT] : "2020/03/30"[PDAT]
Found 1438 results
Going to download record 1 to 10
Added new PMID: 31755794
Added new PMID: 32131996
Added new PMID: 31190616
Added new PMID: 31916895
Added new PMID: 29083658
Added new PMID: 31841260
Added new PMID: 31559860
Added new PMID: 32021405
Added new PMID: 32214840
Added new PMID: 32209533
Going to download record 11 to 20
Added new PMID: 31711967
Added new PMID: 31976778
Added new PMID: 32089833
Added new PMID: 32193022
Added new PMID: 32017742
Added new PMID: 32158008
Added new PMID: 31591925
Added new PMID: 32191688
Added new PMID: 31985627
Added new PMID: 31476423
Going to download record 21 to 30
Added new PMID: 31812604
Added new PMID: 31813472
Added new PMID: 30877807
Added new PMID: 32118751
Added new PMID: 31711352
Added new PMID:

Added new PMID: 32119805
Added new PMID: 32119573
Added new PMID: 32119562
Added new PMID: 32119135
Added new PMID: 32119095
PMID [32118751] exists
Added new PMID: 32116211
Added new PMID: 32116198
Added new PMID: 32116072
Added new PMID: 32115985
Going to download record 291 to 300
Added new PMID: 32115638
Added new PMID: 32115477
Added new PMID: 32115321
Added new PMID: 32115046
Added new PMID: 32114329
Added new PMID: 32113620
Added new PMID: 32113200
Added new PMID: 32112795
Added new PMID: 32111605
Added new PMID: 32111271
Going to download record 301 to 310
Added new PMID: 32110882
Added new PMID: 32110522
Added new PMID: 32110409
Added new PMID: 32110262
Added new PMID: 32110089
Added new PMID: 32109998
Added new PMID: 32109955
Added new PMID: 32109744
Added new PMID: 32109552
Added new PMID: 32109313
Going to download record 311 to 320
Added new PMID: 32109184
Added new PMID: 32109019
Added new PMID: 32108096
Added new PMID: 32108048
Added new PMID: 32107833
Added new PMID: 321

Added new PMID: 32015287
Added new PMID: 32014880
Added new PMID: 32014381
Added new PMID: 32014373
Added new PMID: 32013877
Added new PMID: 32013413
Added new PMID: 32013293
Added new PMID: 32013281
Added new PMID: 32013280
Added new PMID: 32013279
Going to download record 581 to 590
Added new PMID: 32013274
Added new PMID: 32012378
Added new PMID: 32011968
Added new PMID: 32011957
Added new PMID: 32011940
Added new PMID: 32011937
Added new PMID: 32011494
Added new PMID: 32011469
Added new PMID: 32011406
Added new PMID: 32011394
Going to download record 591 to 600
Added new PMID: 32011343
Added new PMID: 32010125
Added new PMID: 32009485
Added new PMID: 32009467
Added new PMID: 32008391
Added new PMID: 32008341
Added new PMID: 32007829
Added new PMID: 32007297
Added new PMID: 32007280
Added new PMID: 32006699
Going to download record 601 to 610
Added new PMID: 32006237
Added new PMID: 32006048
Added new PMID: 32005678
Added new PMID: 32005542
Added new PMID: 32005486
Added new PMID: 3

Added new PMID: 31907240
Added new PMID: 31907000
Added new PMID: 31905233
Added new PMID: 31905072
Added new PMID: 31905067
Added new PMID: 31904839
Added new PMID: 31904685
Added new PMID: 31904442
Added new PMID: 31904397
Added new PMID: 31904310
Going to download record 871 to 880
Added new PMID: 31903581
Added new PMID: 31902846
Added new PMID: 31901578
Added new PMID: 31900970
Added new PMID: 31900964
Added new PMID: 31900614
Added new PMID: 31900537
Added new PMID: 31900410
Added new PMID: 31900073
Added new PMID: 31900054
Going to download record 881 to 890
Added new PMID: 31899936
Added new PMID: 31899693
Added new PMID: 31899157
Added new PMID: 31898822
Added new PMID: 31898283
Added new PMID: 31898130
Added new PMID: 31898123
Added new PMID: 31897469
Added new PMID: 31896338
Added new PMID: 31896337
Going to download record 891 to 900
Added new PMID: 31896130
Added new PMID: 31895187
Added new PMID: 31894609
Added new PMID: 31894606
Added new PMID: 31893348
Added new PMID: 3

Added new PMID: 31654604
Added new PMID: 31654451
Added new PMID: 31653480
PMID [31653394] exists
Added new PMID: 31651743
Added new PMID: 31651435
Added new PMID: 31650532
Added new PMID: 31650431
Added new PMID: 31648374
Added new PMID: 31647279
Going to download record 1161 to 1170
Added new PMID: 31647163
Added new PMID: 31643128
PMID [31643087] exists
Added new PMID: 31639639
Added new PMID: 31638877
Added new PMID: 31637645
Added new PMID: 31634620
Added new PMID: 31631255
Added new PMID: 31630286
Added new PMID: 31629032
Going to download record 1171 to 1180
Added new PMID: 31626971
PMID [31625416] exists
Added new PMID: 31624023
Added new PMID: 31623936
Added new PMID: 31618286
Added new PMID: 31617916
Added new PMID: 31616051
Added new PMID: 31616049
Added new PMID: 31612723
PMID [31611048] exists
Going to download record 1181 to 1190
Added new PMID: 31608431
Added new PMID: 31606310
Added new PMID: 31605328
Added new PMID: 31603562
Added new PMID: 31603348
Added new PMID: 316

...

Found 469 results
Going to download record 1 to 10
PMID [32021405] exists
PMID [32110089] exists
PMID [32005212] exists
PMID [31936517] exists
PMID [32075828] exists
PMID [32215455] exists
PMID [32211867] exists
PMID [32119095] exists
PMID [32184278] exists
PMID [31951640] exists
Going to download record 11 to 20
PMID [31705528] exists
PMID [32132787] exists
PMID [32071185] exists
PMID [32066470] exists
PMID [32021393] exists
PMID [32073401] exists
PMID [32000360] exists
PMID [31918732] exists
PMID [32059572] exists
PMID [32126112] exists
Going to download record 21 to 30
PMID [32021808] exists
PMID [31937665] exists
PMID [32000411] exists
PMID [32181385] exists
PMID [31948487] exists
PMID [32132797] exists
PMID [31978076] exists
PMID [32076421] exists
PMID [32038200] exists
Going to download record 31 to 40
PMID [32076590] exists
PMID [31970561] exists
PMID [32096468] exists
PMID [32021411] exists
PMID [32104061] exists
PMID [31913483] exists
Added new PMID: 32211121
PMID [32184653] e

PMID [32132797] exists
PMID [31993837] exists
PMID [32063838] exists
PMID [32047444] exists
PMID [31992138] exists
PMID [31993783] exists
PMID [31990953] exists
PMID [31990932] exists
PMID [31992344] exists
PMID [32012721] exists
Going to download record 321 to 330
PMID [31985399] exists
PMID [32047450] exists
PMID [31986148] exists
PMID [31988224] exists
PMID [32047772] exists
PMID [31908187] exists
PMID [32021808] exists
PMID [31991652] exists
PMID [31705528] exists
PMID [32140335] exists
Going to download record 331 to 340
PMID [31977882] exists
PMID [31980729] exists
PMID [31978076] exists
PMID [31980020] exists
PMID [32158489] exists
PMID [32038200] exists
PMID [31971968] exists
PMID [32038317] exists
PMID [31941660] exists
PMID [31969671] exists
Going to download record 341 to 350
PMID [31970561] exists
PMID [32064417] exists
PMID [32038315] exists
PMID [32021411] exists
PMID [31969157] exists
PMID [32030195] exists
PMID [31961908] exists
PMID [31964994] exists
PMID [31964664] ex

PMID [32140318] exists
PMID [32202643] exists
PMID [32183790] exists
PMID [32180846] exists
PMID [31937259] exists
PMID [32004349] exists
PMID [31918675] exists
PMID [32194881] exists
PMID [32020565] exists
PMID [32033087] exists
Going to download record 31 to 40
PMID [31936646] exists
PMID [32041538] exists
PMID [32087613] exists
PMID [32197636] exists
PMID [32110679] exists
PMID [32118032] exists
PMID [32066484] exists
PMID [32089760] exists
PMID [31931884] exists
Going to download record 41 to 50
PMID [32113478] exists
PMID [32016589] exists
PMID [32190460] exists
PMID [32070374] exists
Added new PMID: 31731333
PMID [32017684] exists
Added new PMID: 32209112
PMID [31910806] exists
PMID [31900133] exists
PMID [31936616] exists
Going to download record 51 to 60
PMID [32017678] exists
PMID [32126971] exists
PMID [32181442] exists
Added new PMID: 32110959
PMID [32021404] exists
PMID [32158741] exists
PMID [32021407] exists
PMID [31949183] exists
PMID [31922561] exists
PMID [32154098] ex

PMID [31566759] exists
PMID [31535218] exists
PMID [31508873] exists
PMID [31486624] exists
PMID [31471183] exists
PMID [31439388] exists
PMID [31403903] exists
PMID [31335010] exists
PMID [31296116] exists
PMID [31267804] exists
Going to download record 101 to 110
PMID [31255891] exists
PMID [30921802] exists
PMID [30877807] exists
PMID [30860750] exists
PMID [30860711] exists
PMID [30821177] exists
PMID [30252268] exists
PMID [30048311] exists
PMID [30020699] exists
PMID [28722939] exists
"analgesics, opioid"[Pharmacological Action] OR "analgesics, opioid"[MeSH Terms] OR ("analgesics"[All Fields] AND "opioid"[All Fields]) OR "opioid analgesics"[All Fields] OR "opioids"[All Fields] AND mortality AND "2020/01/01"[PDAT] : "2020/03/30"[PDAT]
Found 355 results
Going to download record 1 to 10
PMID [32167567] exists
PMID [32021405] exists
PMID [32133370] exists
PMID [31923197] exists
PMID [32101574] exists
PMID [32075828] exists
PMID [32158255] exists
PMID [31999681] exists
PMID [32066470]

PMID [31998736] exists
PMID [31998210] exists
PMID [31924209] exists
PMID [31923236] exists
PMID [31923197] exists
PMID [31960802] exists
PMID [31936616] exists
PMID [31936517] exists
PMID [31936525] exists
PMID [31923072] exists
Going to download record 301 to 310
PMID [32021393] exists
PMID [31924633] exists
PMID [31918702] exists
PMID [31918740] exists
PMID [31918732] exists
PMID [31915240] exists
PMID [31936338] exists
PMID [31936420] exists
PMID [31910933] exists
PMID [31916383] exists
Going to download record 311 to 320
PMID [32021414] exists
PMID [31914940] exists
PMID [31929239] exists
PMID [31771062] exists
PMID [31936060] exists
PMID [31936049] exists
PMID [32051800] exists
PMID [32021579] exists
Going to download record 321 to 330
PMID [32015694] exists
PMID [31912211] exists
PMID [31998468] exists
PMID [32001906] exists
PMID [32001912] exists
PMID [32001913] exists
PMID [32021141] exists
PMID [31910806] exists
PMID [31935805] exists
PMID [31934520] exists
Going to download 

In [12]:
def MakeTemplate():
    headerslist = []
    for H in ('PMID',
              'Title',
              'Link',
                'JournalName',
                'Date Added',
                'Publication Date',
                'Abstract',
                'Tag'):
        print(H)
        headers = '=ImportJSON("https://raw.githubusercontent.com/tofaquih/opioidsPubGet/master/jsonfiles/opioids.json", "/{}", "noInherit,noTruncate",$A$1)'.format(H)
        headerslist.append(headers)

    headerslist
    with open('../templates/template.csv' ,'w'  , newline='' ) as fp:
        W = csv.writer(fp, delimiter=';')
        W.writerow(headerslist)

In [13]:
MakeTemplate()

PMID
Title
Link
JournalName
Date Added
Publication Date
Abstract
Tag
